# <font face="仿宋">课程说明：

---

## <center> 《2025大模型Agent智能体开发实战》体验课
## <center> DeepSeek R1 Distill高效微调入门实战

### 一、unsloth快速使用入门

#### 1.借助unsloth进行模型推理

In [21]:
from unsloth import FastLanguageModel

- 尝试用unsloth进行LLama模型推理

&emsp;&emsp;首先设置关键参数，并读取模型：

In [22]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = False

> 注，若显存不足，则可以load_in_4bit = True，运行4 bit量化版。

In [23]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./DeepSeek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA H800 PCIe. Max memory: 79.205 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

./DeepSeek-R1-Distill-Llama-8B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


> 在INT4量化情况下，8B模型推理仅需7G左右显存。

此时model就是读取进来的DeepSeek R1 8B蒸馏模型：

In [24]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    

而tokenizer则是分词器：

In [25]:
tokenizer

LlamaTokenizerFast(name_or_path='./DeepSeek-R1-Distill-Llama-8B', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<|finetune_right_pad_id|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	128000: AddedToken("<｜begin▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<｜end▁of▁sentence｜>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True

将模型调整为推理模式：

In [26]:
FastLanguageModel.for_inference(model) 

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    

然后即可和模型进行对话：

In [39]:
question = "请问如何证明根号2是无理数？"

然后这里我们首先需要借助分词器，将输入的问题转化为标记索引：

In [40]:
inputs = tokenizer([question], return_tensors="pt").to("cuda")

In [41]:
inputs

{'input_ids': tensor([[128000,  15225,  57107, 109425, 125544, 102831,  18476,     17,  21043,
          43292,  22649,   9039,  11571]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

最后再带入inputs进行对话

In [42]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

此时得到的回复也是词索引：

In [43]:
outputs

tensor([[128000,  15225,  57107,  ...,  37656,  64531,   9039]],
       device='cuda:0')

同样需要分词器将其转化为文本：

In [44]:
response = tokenizer.batch_decode(outputs)

In [45]:
response

['<｜begin▁of▁sentence｜>请问如何证明根号2是无理数？我记得以前学过的证明方法，可能有几种不同的方式。\n\n首先，我记得有一个方法是假设根号2是有理数，然后推导出矛盾。具体来说，假设根号2等于a/b，其中a和b是互质的正整数，没有公因数，且b不为零。然后，通过平方两边，得到2 = a² / b²，进而得到2b² = a²。这样，a²必须是偶数，因此a也是偶数，设为2k。这样，等式变为2b² = (2k)²，也就是2b² = 4k²，进而得到b² = 2k²，这意味着b²也是偶数，所以b也是偶数。这样，a和b都被2整除，这与我们最初的假设a和b互质矛盾，所以根号2是无理数。\n\n不过，我记得还有另一种方法，可能涉及到假设根号2是有理数，然后将其表示为分数，通过某种方式证明会导致无限递减的矛盾。或者，是否还有其他方法，比如利用二次方程的根的性质，或者其他数学概念？\n\n另外，我还记得可能有一个关于连分数展开的方法，或者是利用代数的方式来证明根号2是无理数。比如，使用费马大定理或者其他定理来证明。\n\n不过，我觉得最直接的方法还是假设根号2是有理数，然后推导出矛盾。因为这样比较简单明了，不需要太多复杂的概念。刚才我已经写出了这个方法的基本步骤，但可能需要更详细地展开，确保每一步都没有问题。\n\n让我再理一理这个过程：\n\n1. 假设根号2是有理数，那么它可以表示为最简分数a/b，其中a和b是互质的正整数，且b ≠ 0。\n\n2. 两边平方得到：2 = a² / b²。\n\n3. 移项得到：2b² = a²。\n\n4. 这意味着a²是2的倍数，因此a也必须是2的倍数，因为平方数的因数都是偶数的因数。所以，设a = 2k，其中k是一个正整数。\n\n5. 将a = 2k代入上式，得到：2b² = (2k)² = 4k²。\n\n6. 两边同时除以2，得到：b² = 2k²。\n\n7. 这意味着b²也是2的倍数，因此b也必须是2的倍数。\n\n8. 但是，根据最初的假设，a和b是互质的，即它们的最大公约数是1。如果a和b都被2整除，那么它们都至少有一个公因数2，这与互质的假设矛盾。\n\n9. 因此，原来的假设根号2是有理数是不成立的，所以根号2是无理数。\n\n这个过程看起来是正确的，没有问题。我还记得另一种方法，可能涉及到数论中的无限

In [46]:
print(response[0])

<｜begin▁of▁sentence｜>请问如何证明根号2是无理数？我记得以前学过的证明方法，可能有几种不同的方式。

首先，我记得有一个方法是假设根号2是有理数，然后推导出矛盾。具体来说，假设根号2等于a/b，其中a和b是互质的正整数，没有公因数，且b不为零。然后，通过平方两边，得到2 = a² / b²，进而得到2b² = a²。这样，a²必须是偶数，因此a也是偶数，设为2k。这样，等式变为2b² = (2k)²，也就是2b² = 4k²，进而得到b² = 2k²，这意味着b²也是偶数，所以b也是偶数。这样，a和b都被2整除，这与我们最初的假设a和b互质矛盾，所以根号2是无理数。

不过，我记得还有另一种方法，可能涉及到假设根号2是有理数，然后将其表示为分数，通过某种方式证明会导致无限递减的矛盾。或者，是否还有其他方法，比如利用二次方程的根的性质，或者其他数学概念？

另外，我还记得可能有一个关于连分数展开的方法，或者是利用代数的方式来证明根号2是无理数。比如，使用费马大定理或者其他定理来证明。

不过，我觉得最直接的方法还是假设根号2是有理数，然后推导出矛盾。因为这样比较简单明了，不需要太多复杂的概念。刚才我已经写出了这个方法的基本步骤，但可能需要更详细地展开，确保每一步都没有问题。

让我再理一理这个过程：

1. 假设根号2是有理数，那么它可以表示为最简分数a/b，其中a和b是互质的正整数，且b ≠ 0。

2. 两边平方得到：2 = a² / b²。

3. 移项得到：2b² = a²。

4. 这意味着a²是2的倍数，因此a也必须是2的倍数，因为平方数的因数都是偶数的因数。所以，设a = 2k，其中k是一个正整数。

5. 将a = 2k代入上式，得到：2b² = (2k)² = 4k²。

6. 两边同时除以2，得到：b² = 2k²。

7. 这意味着b²也是2的倍数，因此b也必须是2的倍数。

8. 但是，根据最初的假设，a和b是互质的，即它们的最大公约数是1。如果a和b都被2整除，那么它们都至少有一个公因数2，这与互质的假设矛盾。

9. 因此，原来的假设根号2是有理数是不成立的，所以根号2是无理数。

这个过程看起来是正确的，没有问题。我还记得另一种方法，可能涉及到数论中的无限递降法，即假设根号2是有理数，然后将其表示为最简分数，平方之后得

至此我们就完成了unsloth模型推理流程。

- 尝试使用unsloth调用Qwen模型

&emsp;&emsp;类似的，我们也可以使用unsloth调用Qwen蒸馏模型

In [15]:
model_qwen, tokenizer_qwen = FastLanguageModel.from_pretrained(
    model_name = "./DeepSeek-R1-Distill-Qwen-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.1.8: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA H800 PCIe. Max memory: 79.205 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

./DeepSeek-R1-Distill-Qwen-7B does not have a padding token! Will use pad_token = <|vision_pad|>.


In [16]:
FastLanguageModel.for_inference(model_qwen) 

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
 

In [48]:
inputs = tokenizer_qwen([question], return_tensors="pt").to("cuda")

outputs = model_qwen.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer_qwen.batch_decode(outputs)

print(response[0])

<｜begin▁of▁sentence｜>请问如何证明根号2是无理数？我记得这个证明要用到无限下降法，或者说归谬法，但具体的步骤我还不太清楚。

嗯，好的，我现在要证明√2是无理数。首先，我需要明确什么是无理数。无理数是指不能表示为两个整数之比的数，也就是无法写成分数形式的数。那根号2是不是这样的数呢？我记得老师上课讲过，用反证法证明的，可能需要用到无限下降法。让我一步一步来思考一下。

首先，假设√2是有理数，也就是说它可以表示为两个互质的整数a和b的比值，也就是√2 = a/b，其中a和b的最大公约数是1，也就是它们互质。

那如果我把这个式子两边平方，得到2 = a² / b²，那么两边同时乘以b²，得到2b² = a²。这说明a²是偶数，因为2b²显然是偶数。那么a必须是偶数吗？因为偶数的平方是偶数，奇数的平方是奇数，所以如果a²是偶数，那么a本身必须是偶数。

既然a是偶数，我们可以把它表示为2k，其中k是一个整数。那把这个代入之前的等式，2b² = a² = (2k)² = 4k²，所以2b² = 4k²，两边同时除以2，得到b² = 2k²。这说明b²是偶数，所以b也是偶数。

但是这里有个问题，我之前假设a和b是互质的，也就是它们的最大公约数是1，但是现在我得到a和b都是偶数，也就是说它们都有2这个公约数，这与它们互质的假设矛盾。所以，这意味着我的初始假设是错误的，也就是说√2不能表示为两个整数的比值，因此√2是无理数。

不过，我是不是哪里漏了？比如，无限下降法是不是应用在这里了？因为当我假设√2是有理数，然后得到了a和b都是偶数，这样我们可以继续分解下去，得到更小的整数，从而形成无限下降的链，这是不可能的，所以最初的假设不成立。

对的，这就是无限下降法的运用，它证明了不存在这样的a和b，所以√2是无理数。这个过程是不是很严谨呢？让我再检查一下。

首先，假设√2 = a/b，其中a和b互质。平方后得到2b² = a²，说明a是偶数，所以设a=2k，代入得到2b² = 4k²，即b²=2k²，说明b也是偶数。那么a和b都有公约数2，这与互质的假设矛盾，所以√2不是有理数，而是无理数。

嗯，看来这个证明是正确的。我是不是应该考虑其他可能性？比如，如果a和b不是互质的，会不会有其他情况？不过，通常在证明中，我们都会把分数化为最简形式，也就是互质的，

> 注，以下实验均以DeepSeek R1 LLama3蒸馏模型为例进行演示和讲解，若想替换为DeepSeek R1 Qwen模型，则可以直接替换模型名称即可。

#### 2.带入问答模板进行回答

- 结构化输入方法

In [50]:
prompt_style_chat = """请写出一个恰当的回答来完成当前对话任务。

### Instruction:
你是一名助人为乐的助手。

### Question:
{}

### Response:
<think>{}"""

In [51]:
question = "你好，好久不见！"

In [52]:
[prompt_style_chat.format(question, "")]

['请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n你好，好久不见！\n\n### Response:\n<think>']

In [53]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [54]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

In [55]:
response = tokenizer.batch_decode(outputs)

In [56]:
response

['<｜begin▁of▁sentence｜>请写出一个恰当的回答来完成当前对话任务。\n\n### Instruction:\n你是一名助人为乐的助手。\n\n### Question:\n你好，好久不见！\n\n### Response:\n<think>\n嗯，用户说“你好，好久不见！”，看起来是想和我打招呼，可能是想聊天或者讨论一些事情。作为助手，我应该友好地回应，表达开心并询问情况。要确保回应自然，避免太正式或太随便。可能的话，鼓励用户继续交流，表现出我的热情和愿意帮助的态度。所以，我会用一个开心的表情符号，表达高兴，并询问用户最近的情况，最后用一个友好的表情符号来结束，保持整体的友好和亲切。\n</think>\n\n你好！很高兴见到你，好久不见哦！最近怎么样？有什么我可以帮忙的吗？ 😊<｜end▁of▁sentence｜>']

In [57]:
print(response[0].split("### Response:")[1])


<think>
嗯，用户说“你好，好久不见！”，看起来是想和我打招呼，可能是想聊天或者讨论一些事情。作为助手，我应该友好地回应，表达开心并询问情况。要确保回应自然，避免太正式或太随便。可能的话，鼓励用户继续交流，表现出我的热情和愿意帮助的态度。所以，我会用一个开心的表情符号，表达高兴，并询问用户最近的情况，最后用一个友好的表情符号来结束，保持整体的友好和亲切。
</think>

你好！很高兴见到你，好久不见哦！最近怎么样？有什么我可以帮忙的吗？ 😊<｜end▁of▁sentence｜>


- 复杂问题测试

In [58]:
question = "请证明根号2是无理数。"

In [59]:
inputs = tokenizer([prompt_style_chat.format(question, "")], return_tensors="pt").to("cuda")

In [60]:
outputs = model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

In [61]:
response = tokenizer.batch_decode(outputs)

In [62]:
print(response[0].split("### Response:")[1])


<think>
嗯，今天老师布置了一个数学作业，让我们证明根号2是无理数。听起来有点难，但我觉得我可以试试看。首先，我得回忆一下什么是有理数和无理数。有理数是可以表示成两个整数相除的数，比如1/2或者3/4。而无理数则不能这样表示，比如根号2、根号3这些。

那么，证明根号2是无理数，我应该从什么角度入手呢？也许可以用反证法。就是假设根号2是有理数，然后推导出矛盾，这样就能证明它其实是无理数了。

好，假设根号2是一个有理数，那它就可以表示成两个整数a/b的形式，其中a和b是没有公因数的整数，而且b不等于零。为了简化，我们可以让a和b互质，也就是说它们的最大公约数是1。

接下来，我可以把这个等式平方两边，得到：
(√2)² = (a/b)²
也就是：
2 = a² / b²
然后，两边同时乘以b²，得到：
2b² = a²
这说明a²是一个整数，而且等于2b²。那么，a²必须是2的倍数，因为b²乘以2的结果是a²。

既然a²是2的倍数，那么a本身也必须是2的倍数。因为只有当a是偶数的时候，a²才会是偶数。所以，我们可以把a表示成2k，其中k是一个整数。代入上面的等式：
(2k)² = 2b²
也就是：
4k² = 2b²
两边同时除以2，得到：
2k² = b²
这意味着b²也是2的倍数，所以b也必须是2的倍数。

但是，根据我们最初的假设，a和b是互质的，意味着它们没有共同的因数。如果a和b都是2的倍数，那么它们至少有一个公因数2，这与互质的假设矛盾。因此，假设根号2是有理数是不成立的，从而证明了根号2是一个无理数。

嗯，整个过程感觉还挺顺利的，虽然一开始有点不太确定用反证法是否正确，但通过代数运算一步步推导，确实找到了矛盾点，说明假设不成立，所以根号2确实是无理数。希望老师会满意这个证明过程。
</think>

根号2是无理数。

**详细证明：**

我们可以使用反证法来证明根号2是无理数。

假设根号2是有理数，那么它可以表示为两个互质的整数a/b，其中b ≠ 0且a和b没有公因数。

1. 将等式两边平方：
   \[
   (\sqrt{2})^2 = \left(\frac{a}{b}\right)^2
   \]
   得到：
   \[
   2 = \frac{a^2}{b^2}
   \]

2. 两边同时乘以b²：
   \[
  

#### 3.原始模型的医疗问题问答

- 重新设置问答模板

In [66]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>{}"""

翻译如下：

```python
prompt_style = """以下是一个任务说明，配有提供更多背景信息的输入。
请写出一个恰当的回答来完成该任务。
在回答之前，请仔细思考问题，并按步骤进行推理，确保回答逻辑清晰且准确。

### Instruction:
您是一位具有高级临床推理、诊断和治疗规划知识的医学专家。
请回答以下医学问题。
```

&emsp;&emsp;接下来我们抽取部分medical-o1-reasoning-SFT数据集中问题进行提问，并查看初始状态下模型回答结果。

In [67]:
question_1 = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"

翻译：一位61岁的女性，有长期在咳嗽或打喷嚏等活动中发生不自主尿液流失的病史，但夜间没有漏尿。她接受了妇科检查和Q-tip测试。根据这些检查结果，膀胱测量（cystometry）最可能会显示她的残余尿量和逼尿肌收缩情况如何？









In [68]:
question_2 = "Given a patient who experiences sudden-onset chest pain radiating to the neck and left arm, with a past medical history of hypercholesterolemia and coronary artery disease, elevated troponin I levels, and tachycardia, what is the most likely coronary artery involved based on this presentation?"

翻译：面对一位突发胸痛并放射至颈部和左臂的患者，其既往病史包括高胆固醇血症和冠状动脉疾病，同时伴有升高的肌钙蛋白I水平和心动过速，根据这些临床表现，最可能受累的冠状动脉是哪一条？

- 问答测试

In [70]:
inputs1 = tokenizer([prompt_style.format(question_1, "")], return_tensors="pt").to("cuda")


outputs1 = model.generate(
    input_ids=inputs1.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response1 = tokenizer.batch_decode(outputs1)

In [71]:
print(response1[0].split("### Response:")[1])


<think>
Okay, so I'm trying to figure out this medical question. Let's break it down step by step. The patient is a 61-year-old woman with a history of involuntary urine loss during activities like coughing or sneezing, but she doesn't leak at night. She's undergoing a gynecological exam and a Q-tip test. The question is about what cystometry would show regarding her residual volume and detrusor contractions.

First, I know that involuntary urine loss during activities like sneezing or coughing is often related to stress urinary incontinence. Stress incontinence usually happens when the urethral muscles aren't strong enough to prevent the bladder from leaking when there's increased pressure, like from coughing.

Now, the Q-tip test. From what I remember, the Q-tip is a urethral catheter used to measure urethral pressure profiles. It's often used to assess urethral function. A positive Q-tip test result, where the urethral pressure is lower than the intravesicular pressure during the V

翻译如下：

\<think>  
好的，我正在尝试分析这个医学问题。我们一步步来分解。患者是一位61岁的女性，有在咳嗽或打喷嚏等活动中发生不自主尿液流失的病史，但她夜间没有漏尿。她正在接受妇科检查和Q-tip测试。问题是关于膀胱测量（cystometry）会显示她的残余尿量和逼尿肌收缩情况。

首先，我知道在像打喷嚏或咳嗽等活动中发生不自主尿液流失通常与压力性尿失禁有关。压力性尿失禁通常发生在尿道肌肉不足以在压力增大的情况下（比如咳嗽时）防止膀胱漏尿时。

接下来是Q-tip测试。根据我记得的，Q-tip是一种用于测量尿道压力曲线的尿道导管。它通常用于评估尿道功能。Q-tip测试阳性结果，即在Valsalva操作过程中尿道压力低于膀胱内压，与内源性括约肌缺陷相关，这是一种压力性尿失禁类型。

由于患者有在活动中出现不自主漏尿的病史，但夜间没有漏尿，更可能是压力性尿失禁，而不是像急迫性尿失禁那样的情况，急迫性尿失禁通常伴有夜间漏尿。因此，如果Q-tip测试阳性，提示内源性括约肌缺陷。

现在，谈到膀胱测量。膀胱测量是一种测试，旨在测量膀胱在充盈过程中的反应以及逼尿肌的收缩情况。它可以显示是否存在膀胱过度活动症（OAB），即引起急迫感和频尿的情况，或是否存在逼尿肌低活动性，导致尿潴留。

在这种情况下，患者的主要问题是压力性尿失禁，这更与无法在压力增大时保持尿液有关。膀胱测量会查看逼尿肌的收缩情况。如果逼尿肌低活动性，它将不能强有力地收缩以排空膀胱，导致残余尿量。但如果逼尿肌过度活跃，可能会收缩过度，导致急迫感。

鉴于患者有压力性尿失禁的病史和Q-tip测试阳性，提示内源性括约肌缺陷，我认为膀胱测量会显示逼尿肌的收缩是正常的。问题不在于逼尿肌收缩的能力，而是无法密封尿道以保持压力。因此，残余尿量可能是正常的，除非有明显的尿潴留，但关键发现是逼尿肌的收缩是正常的，而不是过度活跃。

等等，但会不会有残余尿量？如果患者排尿后膀胱中残留一些尿液，那就是残余尿量。但如果没有尿潴留的症状，比如膀胱饱胀或排尿困难，那么这种情况的可能性较小。主要问题是在活动中发生的尿失禁，因此逼尿肌收缩是正常的，残余尿量在正常范围内，除非有其他情况。

所以，综合来看，膀胱测量可能会显示逼尿肌的收缩正常，残余尿量正常。问题更多是在括约肌方面，而不是逼尿肌。  
\</think>

根据对患者病史和Q-tip测试结果的分析，膀胱测量最可能显示逼尿肌的收缩正常，残余尿量正常。主要问题似乎是由于内源性括约肌缺陷引起的压力性尿失禁，如Q-tip测试阳性所示。这种情况通常影响尿道括约肌在压力增大时防止漏尿的能力，而不是逼尿肌的收缩能力。因此，逼尿肌的收缩并未过度活跃，残余尿量在正常范围内。

标准答案：

在这种压力性尿失禁的情况下，膀胱测压检查（cystometry）最可能显示**正常的排尿后残余尿量**，因为压力性尿失禁通常不会影响膀胱排空功能。此外，由于压力性尿失禁主要与**身体用力**有关，而不是膀胱过度活动症（OAB），因此在测试过程中**不太可能观察到逼尿肌的非自主收缩**。

In [74]:
inputs2 = tokenizer([prompt_style.format(question_2, "")], return_tensors="pt").to("cuda")


outputs2 = model.generate(
    input_ids=inputs2.input_ids,
    max_new_tokens=1200,
    use_cache=True,
)

response2 = tokenizer.batch_decode(outputs2)

In [75]:
print(response2[0].split("### Response:")[1])


<think>
Okay, so I have a patient presenting with sudden-onset chest pain that's radiating to the neck and left arm. That makes me think of a heart attack because those symptoms are classic—angina or myocardial infarction. The left arm pain, neck pain, and sometimes jaw or back pain can all be related to coronary artery issues. 

Looking at the past medical history, the patient has hypercholesterolemia, which is high cholesterol, and coronary artery disease. Those are both risk factors for atherosclerosis, which can lead to blockages in the coronary arteries. The elevated troponin I levels are a big clue because troponin is a cardiac enzyme released when the heart muscle is damaged, which is a sign of a heart attack. Also, the patient is experiencing tachycardia, which means their heart is beating faster than usual. In a heart attack, the heart might beat faster as it tries to pump blood to compensate for the blocked artery.

Now, considering the coronary arteries, the left main coron

翻译如下：

\<think>  
好的，我有一位患者，突然出现胸痛，并放射到颈部和左臂。这让我想到了心脏病发作，因为这些症状很经典——心绞痛或心肌梗死。左臂痛、颈部痛，有时还会伴随下颌或背部的疼痛，这些都可能与冠状动脉问题相关。

从病史来看，患者有**高胆固醇血症**（即高胆固醇）和**冠状动脉疾病**，这两个因素都是**动脉粥样硬化**的风险因素，可能导致冠状动脉发生堵塞。**肌钙蛋白I升高**是一个很大的线索，因为肌钙蛋白是心肌受损时释放的心脏酶，通常表明发生了心肌梗死。另外，患者还出现了**心动过速**，即心跳比平常快。在心肌梗死时，心脏可能会加速跳动，以试图通过增加心脏输出量来补偿被阻塞的冠状动脉。

考虑到冠状动脉，**左主冠状动脉**（LMCA）为整个左侧心脏提供血液，包括左心室，而左心室是一个关键的泵血肌肉。如果这里发生堵塞，可能导致更严重的心肌梗死，因为左心室至关重要。**右冠状动脉**为右心室和左心室下壁提供血液，这里的堵塞也是可能的，但**左主冠状动脉**通常与上述症状更相关，尤其是当肌钙蛋白升高时。

所以，将所有因素综合考虑，最可能受累的冠状动脉是**左主冠状动脉**（LMCA）。患者的病史、肌钙蛋白升高以及典型的胸痛放射症状都指向了这一动脉作为罪魁祸首。

\</think>

最可能受累的冠状动脉是**左主冠状动脉（LMCA）**。

**解释：**
- **症状：** 患者突发胸痛并放射至颈部和左臂，以及肌钙蛋白升高，提示急性冠状动脉综合症，可能是心肌梗死。
- **病史：** 高胆固醇血症和冠状动脉疾病病史是动脉粥样硬化的风险因素，可能导致冠状动脉堵塞。
- **心动过速：** 心率增加可能是心脏为补偿心肌血流减少而产生的反应。
- **冠状动脉考虑：** 左主冠状动脉供应左心室，这个肌肉对心脏功能至关重要。与右冠状动脉相比，左主冠状动脉的堵塞会导致更严重且危及生命的心肌梗死，右冠状动脉通常供应的是不那么关键的区域。

因此，症状、肌钙蛋白升高以及患者的病史强烈指向**左主冠状动脉**（LMCA）作为最可能的罪魁祸首。<｜end▁of▁sentence｜>

标准答案：

根据患者表现出的突然胸痛并放射至颈部和左臂，结合其有高胆固醇血症和冠状动脉疾病的病史，肌钙蛋白升高和心动过速，临床症状强烈提示左前降支（LAD）动脉受累。该动脉通常是引发此类症状的罪魁祸首，因为它供应了心脏的大部分区域。放射性疼痛和肌钙蛋白升高的组合表明心肌受损，这使得LAD成为最可能的致病动脉。然而，在没有进一步的诊断检查（如心电图）的情况下，最终的确诊仍需等待确认。

能够看出，在原始状态下，模型能够进行推理并给出回复，但实际上第一个回答过程并不符合医学规范，而第二个问题则直接回答错误。由此可见，在初始状态下，模型对于medical-o1-reasoning-SFT数据集问答效果并不好。

&emsp;&emsp;接下来尝试进行微调，并测试微调后模型问答效果。

### 二、最小可行性实验

&emsp;&emsp;接下来我们尝试进行模型微调，对于当前数据集而言，我们可以带入原始数据集的部分数据进行微调，也可以带入全部数据并遍历多次进行微调。对于大多数的微调实验，我们都可以从最小可行性实验入手进行微调，也就是先尝试带入少量数据进行微调，并观测微调效果。若微调可以顺利执行，并能够获得微调效果，再考虑带入更多的数据进行更大规模微调。

#### 1.数据集准备

&emsp;&emsp;这里我们直接从huggingface上下载medical-o1-reasoning-SFT数据集。

- 设置代理环境

&emsp;&emsp;由于huggingface网络受限，下载数据集前需要先进行网络环境设置。若是AutoDL服务器，则可以按照如下方式开启学术加速，从而顺利连接huggingface并进行数据集下载：

In [84]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

- 下载数据集

&emsp;&emsp;接下来使用datasets进行数据集下载

In [ ]:
!pip install datasets

In [101]:
import os
from datasets import load_dataset

再次确认提示词模板：

In [90]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

然后提取并设置文本生成结束的标记：

In [92]:
EOS_TOKEN = tokenizer.eos_token
tokenizer.eos_token

'<｜end▁of▁sentence｜>'

然后定义函数，用于对medical-o1-reasoning-SFT数据集进行修改，Complex_CoT列和Response列进行拼接，并加上文本结束标记：

In [88]:
def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206180316919.png" alt="image-20250206180316919" style="zoom:50%;" />

在最小可行性实验中，我们可以只下载500条数据进行微调即可看出效果：

In [85]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train[0:500]",trust_remote_code=True)

Using the latest cached version of the dataset since FreedomIntelligence/medical-o1-reasoning-SFT couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'en' at /root/.cache/huggingface/datasets/FreedomIntelligence___medical-o1-reasoning-sft/en/0.0.0/4c9573e7de1e8660b88158db2efa7c7204bbd269 (last modified on Wed Feb  5 01:06:32 2025).


In [86]:
dataset[0]

{'Question': 'A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?',
 'Complex_CoT': "Okay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her abdominal pressure like coughing or sneezing. This sounds a lot like stress urinary incontinence to me. Now, it's interesting that she doesn't have any issues at night; she isn't experiencing leakage while sleeping. This likely means her bladder's ability to hold urine is fine when she isn't under physical stress. Hmm, that's a clue that we're dealing with something related to pressure rather than a bladder muscle problem. \n\nThe fact that she underwent a Q-tip test is intriguing too. This 

然后进行结构化处理：

In [93]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

将数据集整理为如下形式：

In [94]:
dataset["text"][0]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her ab

- 数据集保存地址

默认情况下数据集保存在主目录下.cache文件夹中，数据文件格式如下所示：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206195216257.png" alt="image-20250206195216257" style="zoom:50%;" />

#### 2.开启微调

&emsp;&emsp;然后即可把模型设置为微调模式：

In [95]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


然后导入相关的库：

In [96]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

创建有监督微调对象：

In [97]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

这段代码主要是用 **`SFTTrainer`** 进行 **监督微调（Supervised Fine-Tuning, SFT）**，适用于 `transformers` 和 `Unsloth` 生态中的模型微调：
**1. 导入相关库**
- **`SFTTrainer`**（来自 `trl` 库）：  
  - `trl`（Transformer Reinforcement Learning）是 Hugging Face 旗下的 `trl` 库，提供 **监督微调（SFT）** 和 **强化学习（RLHF）** 相关的功能。
  - `SFTTrainer` 主要用于 **有监督微调（Supervised Fine-Tuning）**，适用于 `LoRA` 等低秩适配微调方式。

- **`TrainingArguments`**（来自 `transformers` 库）：  
  - 这个类用于定义 **训练超参数**，比如批量大小、学习率、优化器、训练步数等。

- **`is_bfloat16_supported()`**（来自 `unsloth`）：  
  - 这个函数检查 **当前 GPU 是否支持 `bfloat16`（BF16）**，如果支持，则返回 `True`，否则返回 `False`。
  - `bfloat16` 是一种更高效的数值格式，在 **新款 NVIDIA A100/H100** 等 GPU 上表现更优。

**2. 初始化 `SFTTrainer` 进行模型微调**

##### **参数解析**
##### **① `SFTTrainer` 部分**
| 参数 | 作用 |
|------|------|
| `model=model` | 指定需要进行微调的 **预训练模型** |
| `tokenizer=tokenizer` | 指定 **分词器**，用于处理文本数据 |
| `train_dataset=dataset` | 传入 **训练数据集** |
| `dataset_text_field="text"` | 指定数据集中哪一列包含 **训练文本**（在 `formatting_prompts_func` 里处理） |
| `max_seq_length=max_seq_length` | **最大序列长度**，控制输入文本的最大 Token 数量 |
| `dataset_num_proc=2` | **数据加载的并行进程数**，提高数据预处理效率 |

##### **② `TrainingArguments` 部分**
| 参数 | 作用 |
|------|------|
| `per_device_train_batch_size=2` | 每个 **GPU/设备** 的训练批量大小（较小值适合大模型） |
| `gradient_accumulation_steps=4` | **梯度累积步数**（相当于 `batch_size=2 × 4 = 8`） |
| `warmup_steps=5` | **预热步数**（初始阶段学习率较低，然后逐步升高） |
| `max_steps=60` | **最大训练步数**（控制训练的总步数，此处总共约消耗60*8=480条数据） |
| `learning_rate=2e-4` | **学习率**（`2e-4` = 0.0002，控制权重更新幅度） |
| `fp16=not is_bfloat16_supported()` | 如果 **GPU 不支持 `bfloat16`，则使用 `fp16`（16位浮点数）** |
| `bf16=is_bfloat16_supported()` | 如果 **GPU 支持 `bfloat16`，则启用 `bfloat16`（训练更稳定）** |
| `logging_steps=10` | **每 10 步记录一次训练日志** |
| `optim="adamw_8bit"` | **使用 `adamw_8bit`（8-bit AdamW优化器）减少显存占用** |
| `weight_decay=0.01` | **权重衰减（L2 正则化）**，防止过拟合 |
| `lr_scheduler_type="linear"` | **学习率调度策略**（线性衰减） |
| `seed=3407` | **随机种子**（保证实验结果可复现） |
| `output_dir="outputs"` | **训练结果的输出目录** |

然后设置wandb（可选）：

In [98]:
import wandb

In [14]:
wandb.login(key="YOUR_WANDB_API_KEY")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

然后开始微调：

In [38]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,1.941300
20,1.471400
30,1.410600
40,1.320400
50,1.355400
60,1.322700


此时wandb中显示内容如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200441907.png" alt="image-20250206200441907" style="zoom:50%;" />

In [103]:
trainer_stats

TrainOutput(global_step=60, training_loss=1.4551897207895914, metrics={'train_runtime': 117.4543, 'train_samples_per_second': 4.087, 'train_steps_per_second': 0.511, 'total_flos': 1.853060916751565e+16, 'train_loss': 1.4551897207895914, 'epoch': 0.96})

注意，unsloth在微调结束后，会自动更新模型权重（在缓存中），因此无需手动合并模型权重即可直接调用微调后的模型：

In [105]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [106]:
inputs = tokenizer([prompt_style.format(question_1, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

In [107]:
print(response[0].split("### Response:")[1])


<think>
Alright, let's think about this. We have a 61-year-old woman who's been dealing with some pretty frustrating symptoms for a while. She's been involuntarily losing urine when she coughs or sneezes, which sounds like a common issue for people with urinary incontinence, particularly when it comes to things like stress incontinence. But, she doesn't have any leakage at night, which is interesting because that's usually a clue for nocturnal enuresis, but she's not experiencing that. So, we're dealing with a case of stress incontinence here.

Now, she undergoes a gynecological exam and a Q-tip test. Let's break this down. A gynecological exam is pretty standard for checking things like pelvic anatomy and function. The Q-tip test, on the other hand, is specifically designed to assess the function of the urethra and bladder neck. This test involves inserting a Q-tip catheter into the urethra and then having the person cough to see how it moves. If the urethra doesn't move much, it sug

测试第二个问题问答效果：

In [108]:
inputs = tokenizer([prompt_style.format(question_2, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)

In [109]:
print(response[0].split("### Response:")[1])


<think>
Alright, let's think about this. We've got a patient with sudden chest pain, and it's radiating to the neck and left arm. That sounds like angina pectoris. Angina is usually due to reduced blood flow to the heart muscle. Now, I remember that there are different types of angina, and each one relates to a specific coronary artery.

Let's consider what we know. The patient has hypercholesterolemia, which means there's a higher level of cholesterol in their blood. This could lead to atherosclerosis, narrowing the arteries over time. They also have coronary artery disease, which means they have some narrowing in their arteries. So, this makes it more likely that they have significant blockages.

Looking at their symptoms, the pain is radiating to the neck and left arm. This sounds a lot like angina due to atherosclerosis of the left main coronary artery, which supplies the entire left ventricle. This artery is crucial because if it's blocked, it can lead to severe issues like a hea

> 此时模型认为“左主冠状动脉最可能是该患者症状的罪魁祸首”，但实际上应该是“左前降支（LAD）动脉受累”导致。

能够发现，第一个问题回答更加规范，并且回答正确。但第二个问题仍然回答错误。由此可以考虑继续进行大规模微调。不过在此之前，我们可以将现在小规模微调的模型进行本地保存。

#### 3.模型合并

此时本地保存的模型权重在`outputs`文件夹中：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206195427494.png" alt="image-20250206195427494" style="zoom:50%;" />

然后可使用如下代码进行模型权重合并：

In [110]:
new_model_local = "DeepSeek-R1-Medical-COT-Tiny"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 719.9 out of 1007.51 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 57.67it/s]


Unsloth: Saving tokenizer... Done.
Done.


保存结束后，即可在当前文件夹中看到对应模型：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200056475.png" alt="image-20250206200056475" style="zoom:50%;" />

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206200157110.png" alt="image-20250206200157110" style="zoom:50%;" />

然后即可将其推送到huggingface上并保存为GGUF格式文件并进行调用。

In [114]:
# model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q4_k_m")
# model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q8_0")
# model.save_pretrained_gguf("dir", tokenizer, quantization_method = "f16")

### 三、完整高效微调实验

&emsp;&emsp;接下来我们尝试带入全部数据进行高效微调，以提升模型微调效果。

In [18]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [19]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

此时读取全部数据

In [21]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en", split = "train",trust_remote_code=True)
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

Map:   0%|          | 0/25371 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nA 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?\n\n### Response:\n<think>\nOkay, let's think about this step by step. There's a 61-year-old woman here who's been dealing with involuntary urine leakages whenever she's doing something that ups her ab

In [22]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  
    loftq_config=None,
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


这里设置epoch为3，遍历3次数据集：

In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs = 3,
        warmup_steps=5,
        # max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/25371 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 25,371 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 9,513
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.285900
20,1.262500
30,1.245400
40,1.284500
50,1.319800
60,1.298900
70,1.298400
80,1.251900
90,1.295300
100,1.244500


这里总共训练约15个小时。

In [29]:
trainer_stats

TrainOutput(global_step=9513, training_loss=1.0824475168592858, metrics={'train_runtime': 20193.217, 'train_samples_per_second': 3.769, 'train_steps_per_second': 0.471, 'total_flos': 2.7936033274397737e+18, 'train_loss': 1.0824475168592858, 'epoch': 2.9992117294655527})

带入两个问题进行测试，均有较好的回答效果：

In [30]:
question = "A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, what would cystometry most likely reveal about her residual volume and detrusor contractions?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Alright, let's think this through step by step. We've got a 61-year-old woman who's been dealing with involuntary urine loss whenever she does something like coughing or sneezing. That sounds like stress urinary incontinence, which usually means there's some kind of leakage when there's increased pressure in the abdomen. Now, the fact that she doesn't have any leakage at night is interesting. That's a big hint because it suggests that the problem isn't about bladder control or retention, since there's no issue when lying down.

Now, let's consider the Q-tip test. This test is a clever way to check for urethral mobility. If the Q-tip moves a lot, it means the urethra is mobile, which is a typical sign of stress incontinence. So, we're probably looking at a condition where the urethra isn't staying closed under pressure.

Now, if we were to do a cystometry on her, we'd be checking a few things. First, there's the residual volume. Since she's not leaking urine at night, her bladd

In [32]:
question = "Given a patient who experiences sudden-onset chest pain radiating to the neck and left arm, with a past medical history of hypercholesterolemia and coronary artery disease, elevated troponin I levels, and tachycardia, what is the most likely coronary artery involved based on this presentation?"


FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])


<think>
Okay, let's think about this. The patient is having sudden chest pain that's shooting to their neck and left arm. That's pretty classic for something like a myocardial infarction, right? I mean, the pain spreading like that usually points to a big problem in the heart's supply.

Now, considering the heart's coronary arteries, I know the left anterior descending artery, or LAD, is a big player. It's like the main highway for blood to reach the front wall of the heart. If there's a blockage there, it can definitely cause pain that radiates to the neck and arm.

Then, there's the right coronary artery, or RCA, which supplies the right side of the heart and can affect the inferior wall of the heart. But wait, the pain pattern here seems to be more on the left side, so maybe the RCA is less likely.

The patient has hypercholesterolemia and coronary artery disease. These conditions put them at risk for atherosclerosis, which can lead to blockages in the coronary arteries. The LAD is

最后进行模型权重保存：

In [31]:
new_model_local = "DeepSeek-R1-Medical-COT"
model.save_pretrained(new_model_local) 
tokenizer.save_pretrained(new_model_local)

model.save_pretrained_merged(new_model_local, tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 712.42 out of 1007.51 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 57.91it/s]


Unsloth: Saving tokenizer... Done.
Done.


<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/image-20250206201842044.png" alt="image-20250206201842044" style="zoom:50%;" />